In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer, ToktokTokenizer
import re
from nltk import pos_tag
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from string import punctuation
from nltk.stem import WordNetLemmatizer
import os
from spellchecker import SpellChecker
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pyLDAvis
import pyLDAvis.gensim
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import re
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
res = pd.read_csv(r"C:\Users\jduto\OneDrive\Documents\Meesters\Coding\Restaurants_Train_v2.csv")
s = res['Sentence']
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
tokenizer = word_tokenize
custom_spellchecker = SpellChecker()

# Contractions dictionary
contractions_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i had",
    "i'll": "i will",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it had",
    "it'll": "it will",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "must've": "must have",
    "mustn't": "must not",
    "needn't": "need not",
    "oughtn't": "ought not",
    "shan't": "shall not",
    "she'd": "she had",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they had",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we had",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "won't": "will not",
    "would've": "would have",
    "wouldn't": "would not",
    "you'd": "you had",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
    "ambience" : "ambiance",
    "waitress" : "waiter",
    "hostess" : "host"
}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))


def expand_contractions(text, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]

    return contractions_re.sub(replace, text)


def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = tokenizer(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text


def preprocess_text(text):
    text = text.lower()
    text = expand_contractions(text)
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text
s['clean_text'] = s.apply(preprocess_text)
# Function to perform spell correction
def spell_correction(text):
    corrected_text = []
    misspelled_words = custom_spellchecker.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_word = custom_spellchecker.correction(word)
            if corrected_word is not None:
                corrected_text.append(corrected_word)
            else:
                corrected_text.append(word)
        else:
            corrected_text.append(word)
    return ' '.join(corrected_text)

# Apply spell correction to the cleaned text
s['clean_text'] =s['clean_text'].apply(spell_correction)

data = list(s['clean_text'])
bigram = gensim.models.Phrases(data, min_count=20, threshold=100) 
trigram = gensim.models.Phrases(bigram[data], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

f = ["friend","list","man","woman","husband","wife","boyfriend","girlfriend","sake",'lot',"way","choice","minute","def","stuff","fact","app","bet","thing"]
stop_words = nltk.corpus.stopwords.words('english') 
def process_words(texts, stop_words=stop_words, allowed_tags=['NOUN']):
    
    """Convert a document into a list of lowercase tokens, build bigrams-trigrams, implement lemmatization"""
    
    # remove stopwords, short tokens and letter accents 
    texts = [[word for word in simple_preprocess(str(doc), deacc=True, min_len=3) if word not in stop_words] for doc in texts]
    
    # bi-gram and tri-gram implementation
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    
    texts_out = []

    # implement lemmatization and filter out unwanted part of speech tags
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_tags])
    
    # remove stopwords and short tokens again after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc), deacc=True, min_len=3) if word not in stop_words] for doc in texts_out]    
    
    return texts_out
     
    
data_ready = process_words(data)  
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

print('Total Vocabulary Size:', len(id2word))
# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

dict_corpus = {}

for i in range(len(corpus)):
  for idx, freq in corpus[i]:
    if id2word[idx] in dict_corpus:
      dict_corpus[id2word[idx]] += freq
    else:
       dict_corpus[id2word[idx]] = freq

    
dict_df = pd.DataFrame.from_dict(dict_corpus, orient='index', columns=['freq'])
#extension = dict_df[dict_df.freq<5].index.tolist()
#stop_words.extend(extension)

data_ready = process_words(data)
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

print('Total Vocabulary Size:', len(id2word))
     
# Filter out words that occur less than 10 documents, or more than 50% of the documents.

#id2word.filter_extremes(no_below=10, no_above=0.8)

print('Total Vocabulary Size:', len(id2word))

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

from gensim.models import LdaModel

# Specify the path to the Mallet binary

# Train the LDA model using LdaModel with MalletCorpus
ldamodel = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=100
)

words = list(id2word.values())
nums = list(id2word.keys())
stacked = s['clean_text']
big_list_of_sentence_topics = []
for i in stacked.T:
    sentence_topics = []
    tokens = i.split()
    for k in tokens:
        if k in stop_words : continue
        try:
            position = words.index(k)
            key = nums[position]
            sentence_topics.append(key)
        except ValueError: continue
    big_list_of_sentence_topics.append(sentence_topics)

test_sentence = big_list_of_sentence_topics[1]
for i in test_sentence:
    words = nums.index(i)
def get_sentence_topics(sentence):
    topic_list = []
    for i in sentence:
        topic_prob = ldamodel.get_term_topics(word_id=i, minimum_probability = 0.0)
        topic_list.append(topic_prob)
    return topic_list
obj = get_sentence_topics(big_list_of_sentence_topics[1])
test_df = pd.DataFrame(obj)
test_df = pd.DataFrame(test_df.stack())
index_list = []
for i in range(len(test_df)):
    counter = i
    index_list.append(counter)
    counter+=1
#print (index_list)
test_df.index = index_list

def predict_top_topic(obj):
    temp_df = pd.DataFrame(obj)
    temp_df = temp_df.stack()
    categories_list = []
    prob_list = []
    index_list = []
    for i in range(len(temp_df)):
        counter = i
        index_list.append(counter)
        counter+=1
    temp_df.index = index_list
    for i in temp_df.T:
        categories_list.append(i[0])
        prob_list.append(i[1])
    categories_df = pd.DataFrame(categories_list)
    col_name = ['cat']
    categories_df.columns = col_name
    categories_df['probs'] = pd.DataFrame(prob_list)
    categories_df = categories_df.groupby(['cat']).sum()
    top = categories_df.sort_values(by=['probs'], ascending = False)
    top = top.head(1).index[0]
    return top

err_list = []
pred_list = []
for i in range(len(big_list_of_sentence_topics)):
    obj = get_sentence_topics(big_list_of_sentence_topics[i])
    try:
        pred = predict_top_topic(obj)
        pred_list.append(pred)
    except ValueError:
            err_list.append(big_list_of_sentence_topics[i])
            #print('Error at position: ',i)
            pred_list.append(999)

pred_df = pd.DataFrame(pred_list)
counts = pred_df.groupby(0).sum()
col = ['topic']
pred_df.columns = col

a = {}
num_clusters =100  # Adjust the number of clusters as needed
for i in range(num_clusters):
    a[i] = (stacked[pred_df.topic == i])
    
# Text Summarization
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Convert the sentences in a0 to a single string


def generate_summary_from_text(text, top_n=5):
    sentences = sent_tokenize(text)
    
    # Preprocess the sentences
    sentences = [sentence.lower() for sentence in sentences]
    sentences = [word_tokenize(sentence) for sentence in sentences]
    sentences = [[word for word in sentence if word.isalnum()] for sentence in sentences]
    
    # Remove stop words
    sentences = [[word for word in sentence if word not in stop_words] for sentence in sentences]
    
    # Calculate sentence scores based on word frequencies
    word_frequencies = {}
    for sentence in sentences:
        for word in sentence:
            if word not in word_frequencies:
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    
    # Normalize the word frequencies
    maximum_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / maximum_frequency
    
    # Calculate sentence scores
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        sentence_scores[i] = 0
        for word in sentence:
            if word in word_frequencies:
                sentence_scores[i] += word_frequencies[word]
    
    # Sort the sentence scores and get the top N sentences
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:top_n]
    top_sentences = [sentences[i] for i in top_sentences]
    
    # Flatten the top sentences and join them to form the summary
    summary = ". ".join([" ".join(sentence) for sentence in top_sentences])
    
    return summary

stop_words = set(stopwords.words('english')) 

def generate_one_word_summary_from_text(text):
    sentences = sent_tokenize(text)
    
    # Preprocess the sentences
    preprocessed_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        tagged_words = pos_tag(words)  # Perform POS tagging
        nouns = [word for word, pos in tagged_words if pos.startswith('N')]  # Retain only nouns
        filtered_words = [word for word in nouns if word.isalnum() and word not in stop_words]
        preprocessed_sentences.append(filtered_words)
    
    # Calculate sentence scores based on word frequencies
    word_frequencies = {}
    for sentence in preprocessed_sentences:
        for word in sentence:
            if word not in word_frequencies:
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    
    # Check if word_frequencies dictionary is empty
    if not word_frequencies:
        return None
    
    # Sort the word frequencies and select the most frequent word
    top_word = max(word_frequencies, key=word_frequencies.get)
    
    return top_word

# Iterate through a0_ to a9_ and generate one-word summaries
summary = []
for i in range(num_clusters):
    set_name = a[i]
    reviews_set = set_name 
    text = ". ".join(reviews_set)
    summary.append(generate_one_word_summary_from_text(text))
    
topic_summary_map = {}

# Iterate over the range of available summaries
for i in range(min(len(summary), 100)):
    topic_summary_map[i] = summary[i]




c:\Users\jduto\AppData\Local\anaconda3\envs\Master\lib\site-packages\google\rpc\__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
c:\Users\jduto\AppData\Local\anaconda3\envs\Master\lib\site-packages\pkg_resources\__init__.py:2350: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)
C:\Users\jduto\AppData\Local\Temp\ipykernel_13244\1321584916.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

Total Vocabulary Size: 1308
Total Vocabulary Size: 1308
Total Vocabulary Size: 1308


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jduto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Specify the file path where you want to save the text file
output_file_path = "data_ready.txt"

# Open the file in write mode and write each processed document on a new line
with open(output_file_path, "w", encoding="utf-8") as output_file:
    for document in data_ready:
        processed_text = " ".join(document)  # Join the words in the document list
        output_file.write(processed_text + "\n")  # Write the processed text to the file

print("Data has been exported to", output_file_path)


Data has been exported to data_ready.txt


In [2]:
test =pd.DataFrame(res['Sentence'])
test['Aspect'] = res['Aspect Term']
test['Aspect Pred'] = pred_df['topic'].map(topic_summary_map)


In [5]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Function to perform lexicon-based sentiment analysis
def perform_sentiment_analysis(text):
    sentiment_score = sia.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    if compound_score > 0:
        return 'positive'
    elif compound_score < 0:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis to the 'clean_text' column
s['sentiment'] = s['clean_text'].apply(perform_sentiment_analysis)

test['polarity'] = res['polarity']
test['predicted_polarity'] = s['sentiment']

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(res['Sentence'], res['polarity'], test_size=0.25, random_state=42)

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert text into a numerical representation
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the sentiment analysis model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train_encoded)

X_vectorised = vectorizer.transform(res['Sentence'])
# Make predictions on the testing set
y_pred = model.predict(X_vectorised)

# Decode the predicted labels
y_pred_decoded = label_encoder.inverse_transform(y_pred)

ytest = res['polarity']
# Evaluate the model's performance
accuracy = accuracy_score(ytest, y_pred_decoded)
precision = precision_score(ytest, y_pred_decoded, average='weighted')
recall = recall_score(ytest, y_pred_decoded, average='weighted')
f1 = f1_score(ytest, y_pred_decoded, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.7971838613593285
Precision: 0.7900800328185226
Recall: 0.7971838613593285
F1-score: 0.7848975479934776


In [7]:
test['polarity'] = res['polarity']
test['predicted_polarity'] = y_pred_decoded

In [8]:
# Assuming 'true_labels' contains the true labels for the data
true_labels = test['Aspect']

# Assuming 'pred_df' contains the predicted topics for each sentence
pred_labels = test['Aspect Pred'].astype(str)

# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_labels)

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

# Print the performance metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


print("SENTIMENT:")
true_labels = test['polarity'] 
# Assuming 'pred_df' contains the predicted topics for each sentence
pred_labels = test['predicted_polarity']

# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_labels)

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

# Print the performance metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.23422691578662333
Precision: 0.008048237293117367
Recall: 0.0189479056526578
F1-score: 0.009604864946259718
SENTIMENT:
Accuracy: 0.7971838613593285
Precision: 0.7601756934504733
Recall: 0.5828866952191635
F1-score: 0.6145919192812342


c:\Users\jduto\AppData\Local\anaconda3\envs\Master\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jduto\AppData\Local\anaconda3\envs\Master\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:

aspect_term_counts = test['Aspect'].apply(preprocess_text).str.lower().value_counts()

top_aspect_terms = aspect_term_counts.head(50).index.tolist()
# Filter the original DataFrame to include only rows with top aspect terms
test_filter = pd.DataFrame(test)
test_filter = test[test['Aspect'].isin(top_aspect_terms)]
test_filter['Aspect Pred'] = test['Aspect Pred']

# Assuming 'true_labels' contains the true labels for the data
true_labels = test_filter['Aspect']

# Assuming 'pred_df' contains the predicted topics for each sentence
pred_labels = test_filter['Aspect Pred'].astype(str)

# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_labels)

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

# Print the performance metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.5334190231362468
Precision: 0.27595455872551666
Recall: 0.21386050877587828
F1-score: 0.21478087065335583


C:\Users\jduto\AppData\Local\Temp\ipykernel_10400\3715440498.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_filter['Aspect Pred'] = test['Aspect Pred']
c:\Users\jduto\AppData\Local\anaconda3\envs\Master\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jduto\AppData\Local\anaconda3\envs\Master\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 

In [10]:
true_labels = test_filter['polarity']
# Assuming 'pred_df' contains the predicted topics for each sentence
pred_labels = test_filter['predicted_polarity']
# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_labels)

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

# Print the performance metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.7718508997429306
Precision: 0.726582491325539
Recall: 0.53990189622392
F1-score: 0.5566340850577435


In [11]:
# Grouping by 'Aspect' and calculating polarity percentages
grouped = test.groupby('Aspect Pred')['predicted_polarity'].value_counts(normalize=True).unstack().fillna(0) * 100
grouped_correct = test.groupby('Aspect')['polarity'].value_counts(normalize=True).unstack().fillna(0) * 100
# Reordering columns for better readability
grouped = grouped[['negative', 'neutral', 'positive']]
grouped_correct = grouped_correct[['negative', 'neutral', 'positive']]
summary = pd.DataFrame(grouped)
summary_c = pd.DataFrame(grouped_correct)
summary.to_excel('summary_pred.xlsx')
summary_c.to_excel('summary_correct.xlsx')

In [12]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(res['Sentence'], res['polarity'], test_size=0.25, random_state=42)

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert text into a numerical representation
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the sentiment analysis model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train_encoded)

# Make predictions on the testing set
y_pred = model.predict(X_test_vectorized)

# Decode the predicted labels
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_decoded)
precision = precision_score(y_test, y_pred_decoded, average='weighted')
recall = recall_score(y_test, y_pred_decoded, average='weighted')
f1 = f1_score(y_test, y_pred_decoded, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.7218614718614719
Precision: 0.6912715775683084
Recall: 0.7218614718614719
F1-score: 0.7003538265684714


In [13]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(res['Sentence'], res['polarity'], test_size=0.25, random_state=42)

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert text into a numerical representation
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the sentiment analysis model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train_encoded)

# Make predictions on the testing set
y_pred = model.predict(X_test_vectorized)

# Decode the predicted labels
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_decoded)
precision = precision_score(y_test, y_pred_decoded, average='weighted')
recall = recall_score(y_test, y_pred_decoded, average='weighted')
f1 = f1_score(y_test, y_pred_decoded, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.7218614718614719
Precision: 0.6912715775683084
Recall: 0.7218614718614719
F1-score: 0.7003538265684714


In [16]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(revi['Review'], revi['Sentiment'], test_size=0.2, random_state=42)
# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert text into a numerical representation
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the sentiment analysis model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train_encoded)

# Make predictions on the testing set
y_pred = model.predict(X_test_vectorized)

# Decode the predicted labels
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_decoded)
precision = precision_score(y_test, y_pred_decoded, average='weighted')
recall = recall_score(y_test, y_pred_decoded, average='weighted')
f1 = f1_score(y_test, y_pred_decoded, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)



Accuracy: 0.9173986170809194
Precision: 0.9061528407823294
Recall: 0.9173986170809194
F1-score: 0.9111622060047108


In [17]:
#label_encoder = LabelEncoder()
#label_encoder.fit(test['polarity'])

# Preprocess the new dataset
X_new = test['Sentence']
y_new_encoded = test['polarity']

# Convert text into a numerical representation
X_new_vectorized = vectorizer.transform(X_new)

# Make predictions on the new dataset
y_new_pred = model.predict(X_new_vectorized)

# Optionally, decode the predicted labels if necessary
y_new_pred_decoded = label_encoder.inverse_transform(y_new_pred)

# Evaluate the model's performance on the new dataset
accuracy_new = accuracy_score(test['polarity'], y_new_pred_decoded)
precision_new = precision_score(test['polarity'], y_new_pred_decoded, average='weighted')
recall_new = recall_score(test['polarity'], y_new_pred_decoded, average='weighted')
f1_new = f1_score(test['polarity'], y_new_pred_decoded, average='weighted')

print("Accuracy on the new dataset:", accuracy_new)
print("Precision on the new dataset:", precision_new)
print("Recall on the new dataset:", recall_new)
print("F1-score on the new dataset:", f1_new)


Accuracy on the new dataset: 0.5778499864608719
Precision on the new dataset: 0.5401115572561377
Recall on the new dataset: 0.5778499864608719
F1-score on the new dataset: 0.5573695962783218


c:\Users\jduto\AppData\Local\anaconda3\envs\Master\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
